1. Install huggingface `transformers` library

In [1]:
!pip install transformers

     |████████████████████████████████| 573kB 3.5MB/s 
     |████████████████████████████████| 3.7MB 50.9MB/s 
     |████████████████████████████████| 1.0MB 40.3MB/s 
     |████████████████████████████████| 890kB 42.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=20c0fcf8e08644aa93a0438dc1e37b39fd518128f044333a5b6f800bffa8d0d3
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


2. Use `BertForQuestionAnswering` from the transformers library and load the pre-trained model `'bert-large-uncased-whole-word-masking-finetuned-squad'`

In [2]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

3. User `BertTokenizer` and load pre-trained tokenizer `'bert-large-uncased-whole-word-masking-finetuned-squad'`

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

4. Define a function `answer_question(question, anwer_title, answer_text)`

In [0]:
import torch
def answer_question(question, answer_title, answer_text):
    '''
    Takes a `question` string, an `answer_title` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair. Limit the maximum numbers of input tokens to 500.
    input_ids = tokenizer.encode(question, answer_text, max_length= 500)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    # Choose the answers without token [SEP] and [CLS] and with at least 3 words. Print out the question, the answer and the corresponding title.
    if "[SEP]" not in answer and "[CLS]" not in answer and len(answer.split()) >= 3:
      print('Question: "' + question + '"')
      print('Answer: "' + answer + '"')
      print('Title: "' + answer_title + '"')
      print('\n')

5. Load `"metadat.csv"`

In [5]:
import pandas as pd

data = pd.read_csv("metadata.csv")

data

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51073,4xhcamks,e14e7f6ffed1a74954167e4fb52d1b2f156171cd,Elsevier,Prevalence of antibodies against transmissible...,10.1016/j.vas.2018.11.003,PMC7104163,NaN,els-covid,Abstract Transmissible gastroenteritis (TGE) i...,2019-06-30,"Valkó, Anna; Bálint, Ádám; Bozsa, Ágnes; Cságo...",Veterinary and Animal Science,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.vas.2018.11.003
51074,z2rqz3jx,9550137f5e105d2dbc14d91659a8c3cdd815c9b1,Elsevier,Can we increase public awareness without creat...,10.1016/j.pec.2013.10.023,PMC7119035,24284163.0,els-covid,NaN,2014-02-28,"Al Turki, Yousef Abdullah",Patient Education and Counseling,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.pec.2013.10.023
51075,wncuc903,2b3a3e3956afd65a8fa11d3867da3e58f8915514,Elsevier,Fast degrading polyesters as siRNA nano-carrie...,10.1016/j.jconrel.2008.06.010,PMC7125568,18619502.0,els-covid,Abstract A potential siRNA carrier for pulmona...,2008-12-18,"Nguyen, Juliane; Steele, Terry W.J.; Merkel, O...",Journal of Controlled Release,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.jconrel.2008.06.010
51076,4x1ckr2i,a67dae6e5f2b58a6f16aaed4011c694f9e64dd88,Elsevier,eIF3f: A central regulator of the antagonism a...,10.1016/j.biocel.2013.06.001,PMC7108353,23769948.0,els-covid,Abstract The eukaryotic initiation factor 3 su...,2013-10-31,"Sanchez, Anthony M.J.; Csibi, Alfredo; Raibon,...",The International Journal of Biochemistry & Ce...,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.biocel.2013.06.001


6. Add the questions in the task into `question_list`

In [0]:
question_list = ["What do we know about virus genetics, origin, and evolution?",
                 "What do we know about the virus origin and management measures at the human-animal interface?"]

7. Run the function `answer_question` to find the answers in the abstract of the `metadata.csv`

In [7]:
# Due the time complexity, we run the answer_question function through the first 100 abstracts in 'metadata.csv'
for question in question_list:
  for i in range(100):
    if pd.isnull(data['abstract'][i]):
      continue
    answer_question(question, data['title'][i], data['abstract'][i])

Question: "What do we know about virus genetics, origin, and evolution?"
Answer: "genetics , origin , and evolution"
Title: "Descriptive review of geographic mapping of severe acute respiratory syndrome (SARS) on the Internet"


Question: "What do we know about virus genetics, origin, and evolution?"
Answer: "an essential precondition for this mode of transmission is the presence of hcv in the nasal secretions of intranasal drug users . methods : blood and nasal secretion samples were collected from five plasma - positive chronic intranasal drug users and tested for hcv rna using rt - pcr . results : hcv was detected in all five blood samples and in the nasal secretions of the subject with the highest serum viral load . conclusions : this study is the first to demonstrate the presence of hcv in nasal secretions ."
Title: "Detection of hepatitis C virus in the nasal secretions of an intranasal drug-user"


Question: "What do we know about virus genetics, origin, and evolution?"
Answer: 

In [7]:
# Randomly sample 1000 rows from dataset
data_random_1000 = data.sample(n = 1000, random_state = 1)
data_random_1000

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
18848,phlxsez8,NaN,PMC,Severe Acute Respiratory Syndrome (SARS) Coron...,10.1128/jvi.01048-15,PMC4580176,26269185.0,unk,Despite the identification of horseshoe bats a...,2015-08-12,"Lau, Susanna K. P.; Feng, Yun; Chen, Honglin; ...",Journal of Virology,NaN,NaN,False,False,NaN,https://jvi.asm.org/content/jvi/89/20/10532.fu...
13874,h5qf2qld,83c206ed163d6024781f1b508c3b536721d69d63,PMC,Peroral immunotheraphy with yolk antibodies fo...,10.1385/ir:21:1:1,PMC7090601,10803878.0,no-cc,Oral administration of specific antibodies is ...,2000,"Carlander, David; Kollberg, Hans; Wejåker, Per...",Immunol Res,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
3433,y33s6ur3,40886c8a1c173dab7ef1685c544a7554f63bd6ff,PMC,Mechanisms and impact of the frequent exacerba...,10.1186/1741-7015-11-181,PMC3750926,23945277.0,cc-by,Exacerbations of chronic obstructive pulmonary...,2013-08-14,"Wedzicha, Jadwiga A; Brill, Simon E; Allinson,...",BMC Med,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
24225,yhpyqya7,4a147480f5801aeaffec786c5e275c8f4eb033c3,PMC,"7 Hoesten, van symptoom tot diagnose",10.1007/978-90-313-6599-9_7,PMC7121593,NaN,no-cc,"Hoesten behoort, naast klachten van het houdin...",2009-10-16,"Sachs, A. P. E.; Verheij, Th. J. M.; Lammers, ...",Longziekten,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
12179,t2iynltq,a29854dea098c27aed6012be2e0c3efa740765cd,PMC,Use of airway clearance therapy in children ho...,10.4102/sajp.v76i1.1367,PMC7059502,32161827.0,cc-by,BACKGROUND: Little is known about the prescrip...,2020-02-19,"Corten, Lieselotte; Morrow, Brenda M.",S Afr J Physiother,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,7s6283y5,fd5e3020671499cde830bd25de618009ece0af13,PMC,Antiviral and Neuroprotective Role of Octaguan...,10.1371/journal.pntd.0000892,PMC2990691,21124882.0,cc-by,"BACKGROUND: Japanese encephalitis (JE), caused...",2010-11-23,"Nazmi, Arshed; Dutta, Kallol; Basu, Anirban",PLoS Negl Trop Dis,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
13230,v60k0qa2,d24b9ee5f6da8aad4daaa035c86598c1f29c19d5,PMC,Clinical and laboratory characteristics of vir...,10.1007/s00508-015-0843-2,PMC7087898,26373742.0,no-cc,BACKGROUND: Viral lower respiratory tract infe...,2015-09-15,"Berce, Vojko; Unuk, Sibila; Duh, Darja; Homšak...",Wien Klin Wochenschr,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
9392,d1k3myo5,90570e69d4ac4b2e72385123ba09f6b754707b61,PMC,Extra Dose of Vitamin C Based on a Daily Suppl...,10.1155/2018/1837634,PMC6057395,30069463.0,cc-by,AIM: To investigate whether vitamin C is effec...,2018-07-05,"Ran, Li; Zhao, Wenli; Wang, Jingxia; Wang, Hon...",Biomed Res Int,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...
25540,z4mgtb6s,b4e03c82e343ee7d2b37096dbac5fbf84d783aa5,Elsevier,Evidence for an RNA pseudoknot loop-helix inte...,10.1006/jmbi.1999.2689,NaN,10329145.0,els-covid,Abstract RNA pseudoknots are structural elemen...,1999-05-07,"Liphardt, Jan; Napthine, Sawsan; Kontos, Harry...",Journal of Molecular Biology,NaN,NaN,True,False,custom_license,https://doi.org/10.1006/jmbi.1999.2689


In [14]:
# Due the time complexity, we run the answer_question function through the random sampled 1000 abstracts in 'metadata.csv'
for question in question_list:
  for i in range(len(data_random_1000)):
    if pd.isnull(data_random_1000['abstract'].iloc[i]):
      continue
    answer_question(question, data_random_1000['title'].iloc[i], data_random_1000['abstract'].iloc[i])

Question: "What do we know about virus genetics, origin, and evolution?"
Answer: "de hoogste incidentie - en prevalentiecijfers"
Title: "7 Hoesten, van symptoom tot diagnose"


Question: "What do we know about virus genetics, origin, and evolution?"
Answer: "our model suggests that the basic reproduction number associated with the outbreak ( at time of writing ) may range from 2 . 0 to 3 . 1 . though these estimates are preliminary and subject to change , they are consistent with previous findings regarding the transmissibility of the related sars - coronavirus and indicate the possibility of epidemic potential"
Title: "Early Transmissibility Assessment of a Novel Coronavirus in Wuhan, China"


Question: "What do we know about virus genetics, origin, and evolution?"
Answer: "history , etiology , epidemiology , clinical signs , pathology , diagnosis , and control"
Title: "Chapter 14 Viral Diseases"


Question: "What do we know about virus genetics, origin, and evolution?"
Answer: "detec